In [1]:
#IMPORTS

import matplotlib
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
import scipy.interpolate
import mne
import scipy
import statistics
from pywt import wavedec
from scipy import signal
import matplotlib.colors as mcolors
import random
import seaborn as sns

In [2]:
#DEF

'''FOR THE INITIAL DATA TREATMENT'''
#DEF UPPER CHANNEL
def upperchanel(chanlesnames):
    upperchanels=[]
    for i in chanlesnames:
        r = i.upper()
        upperchanels.append(r)
    return upperchanels

#DEF GRUP INFO DE CADA COMPONENT
def group_inf(nomgroup, data_chan, upperchanels):
    group_date = []

    for e in nomgroup:
        if e in upperchanels:
            num = upperchanels.index(e)
            #print(num)
            group_date.append(data_chan[num])  
    #print(group_date)
    return group_date

#DEF MEDIUM DELS GRUPS
#group_dat = [[1,2,3,..., 119345],[1,2,3,..., 119345],[1,2,3,..., 119345],[1,2,3,..., 119345]]
def mediumchanels(group_dat):
    segundo = len(group_dat[0])
    primero = len(group_dat)
    mean_data = []
    for i in range (0,segundo):
        suma1=[]
        for e in range(0,primero):
            suma1.append(group_dat[e][i])
        numpero = statistics.median(suma1)
        mean_data.append(numpero)
    return mean_data



def opteciogrups(chanlesnames, groupp, data_chan):
    group_date_finale = [] #Save all informatio chanels 12.
    group_date_final = [] #final copy
    upchan = upperchanel(chanlesnames)
    for i in groupp:
        group_date = group_inf(i, data_chan, upchan)
        mean_group = mediumchanels(group_date)
        mean_array = np.array(mean_group)
        group_date_finale.append(mean_array)
    group_date_final = np.array(group_date_finale)
    return group_date_final



In [3]:
'''FOR THE EPOCH'''
#TO FIND THE NEAREST POSITION OF THE DATA THAT CORESPOND TO THE ONE INTRODUCED.
def find_nearest(array,value): 
       idx = (np.abs(array-value)).argmin()
       return int(idx)

#EXTRACT THE t FROM THE DATA
def grabt(dat):
    Fs = 250.0
    Ts = 1.0/Fs
    t = np.arange(len(dat)) / Fs
    return t

In [4]:
#Init. info.
#GROUPS
group1= ["FP2", "AFZ", "FZ", "FCZ", "CZ"]
group2= ["F7", "F5", "F3", "F1", "AF7", "AF3", "FP1"]
group3= ["FP2", "AF4", "AF8", "F2", "F4", "F6", "F8"]
group4= ["FT7", "T7", "TP7"]
group5= ["FC5", "FC3", "FC1", "C5", "C3", "C1"]
group6= ["CP5", "CP3", "CP1", "P7", "P5", "P3", "P1"]
group7= ["PO7", "PO3", "O1", "PO9"]
group8= ["PO4", "PO8", "O2", "PO10"]
group9= ["P2", "P4", "P6", "P8", "CP2", "CP4", "CP6"]
group10= ["FT8", "T8", "TP8"]
group11= ["CPZ", "PZ", "POZ", "OZ", "IZ"]
group12= ["FC2", "FC4", "FC6", "C2", "C4", "C6"]

#X
groups = [group1, group2, group3, group4, group5, group6, group7, group8, group9, group10, group11, group12]

#General aspects
titlegrup = ["ch1","ch2","ch3","ch4","ch5","ch6","ch7","ch8","ch9","ch10","ch11","ch12"]
ct_ty = ["eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg"]
sfreq = 250
info = mne.create_info(ch_names=titlegrup, sfreq=sfreq, ch_types=ct_ty)

In [19]:
#Load
basePATH = 'Files/Preprocessed/'
listPATH = []
import os

for root, dirs, files in os.walk("Files/Preprocessed/"):
    for filename in files:
        if '.set' in filename:
            listPATH.append(filename)
sorted_list = sorted(listPATH)


In [40]:
total_eo_ec = []
number = 10000
for e in range(0,100):
    for i in sorted_list:
        eo_ec = []
        if str(number) in str(i):
            if 'EC' in str(i):
                eo_ec.append(i)
            if 'EO' in str(i):
                eo_ec.append(i)
    total_eo_ec.append(eo_ec)
    number=number+1

print(total_eo_ec)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['sub-010023_EO.set'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [29]:
number = 10000

for i in sorted_list:
    print('Nom: '+str(i))
    if str(number) in str(i):
        if 'EC' in str(i):
            #LOAD EC
            xEC=mne.io.read_raw_eeglab(basePATH+i, preload=True, verbose=True)
            #GET DATA
            dataEC = xEC._data
            #GET CHANELS
            chanles_namesEC = xEC.ch_names
            #REDUCE CHANELS TO 12
            groups_date_finalleEC = opteciogrups(chanles_namesEC, groups, dataEC)
            #TRANSFORM TO MNE
            rawEC = mne.io.RawArray(groups_date_finalleEC, info)
            print('si '+str(i))
        if 'EO' in str(i):
            #LOAD EO
            xEO=mne.io.read_raw_eeglab(basePATH+i, preload=True, verbose=True)
            #GET DATA
            dataEO = xEO._data
            #GET CHANELS
            chanles_namesEO = xEO.ch_names
            #REDUCE CHANELS TO 12
            groups_date_finalleEO = opteciogrups(chanles_namesEO, groups, dataEO)
            #TRANSFORM TO MNE
            rawEO = mne.io.RawArray(groups_date_finalleEO, info)
            print('si '+str(i))
    number=number+1
        

Nom: sub-010002_EC.set
Nom: sub-010002_EO.set
Nom: sub-010003_EC.set
Nom: sub-010003_EO.set
Reading Files/Preprocessed/sub-010003_EO.fdt
Reading 0 ... 119999  =      0.000 ...   479.996 secs...
Creating RawArray with float64 data, n_channels=12, n_times=120000
    Range : 0 ... 119999 =      0.000 ...   479.996 secs
Ready.
si sub-010003_EO.set
Nom: sub-010004_EC.set
Reading Files/Preprocessed/sub-010004_EC.fdt
Reading 0 ... 119277  =      0.000 ...   477.108 secs...
Creating RawArray with float64 data, n_channels=12, n_times=119278
    Range : 0 ... 119277 =      0.000 ...   477.108 secs
Ready.
si sub-010004_EC.set
Nom: sub-010004_EO.set
Nom: sub-010005_EC.set
Nom: sub-010005_EO.set
Nom: sub-010006_EC.set
Nom: sub-010006_EO.set
Nom: sub-010007_EC.set
Nom: sub-010007_EO.set
Nom: sub-010010_EC.set
Nom: sub-010010_EO.set
Nom: sub-010012_EC.set
Nom: sub-010012_EO.set
Nom: sub-010016_EC.set
Reading Files/Preprocessed/sub-010016_EC.fdt
Reading 0 ... 119042  =      0.000 ...   476.168 secs...

In [23]:
sorted_list

['sub-010002_EC.set',
 'sub-010002_EO.set',
 'sub-010003_EC.set',
 'sub-010003_EO.set',
 'sub-010004_EC.set',
 'sub-010004_EO.set',
 'sub-010005_EC.set',
 'sub-010005_EO.set',
 'sub-010006_EC.set',
 'sub-010006_EO.set',
 'sub-010007_EC.set',
 'sub-010007_EO.set',
 'sub-010010_EC.set',
 'sub-010010_EO.set',
 'sub-010012_EC.set',
 'sub-010012_EO.set',
 'sub-010016_EC.set',
 'sub-010016_EO.set',
 'sub-010017_EC.set',
 'sub-010017_EO.set',
 'sub-010019_EC.set',
 'sub-010019_EO.set',
 'sub-010020_EC.set',
 'sub-010020_EO.set',
 'sub-010021_EC.set',
 'sub-010021_EO.set',
 'sub-010022_EC.set',
 'sub-010022_EO.set',
 'sub-010023_EC.set',
 'sub-010023_EO.set']